In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 13
- The output is far worse than the best corner values (~ −55.8).
- Even with fairly high values in dimensions 3 and 4, performance collapses when dimensions 1 and 2 are moderate.
- This reinforces the finding that all dimensions must be jointly high to achieve strong performance.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm
# --- Original dataset (30 initial points) ---

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
X_init = datain
y_init = dataout

# --- Previous high-performing points ---
X_week1 = np.array([[0.999999, 0.999999, 0.910203, 0.738695]])  # week 1
y_week1 = np.array([-43.88421604431158])

X_week2 = np.array([[0.999999, 0.999999, 0.999999, 0.956184]])        # week 2
y_week2 = np.array([-53.93723834447932])

X_week3 = np.array([[0.990000, 0.990000, 0.990000, 0.990000]])        # week 3
y_week3 = np.array([-53.590344373442626])

X_week4 = np.array([[0.499999, 0.500000, 0.500000, 0.499999]])        # week 4
y_week4 = np.array([-3.9857608859098153])

X_week5 = np.array([[0.999999, 0.999999, 0.999999, 0.960184]])        # week 5
y_week5 = np.array([-54.03075969006948])

X_week6 = np.array([[0.999999, 0.999999, 0.999999, 0.966000]])        # week 6
y_week6 = np.array([-54.191444279951604])

X_week7 = np.array([[0.999999, 0.999999, 0.999999, 0.972000]])        # week 7
y_week7 = np.array([-54.393239979988984])

X_week8 = np.array([[0.999999, 0.999999, 0.999999, 0.990000]])        # week 8
y_week8 = np.array([-55.23213675850386])

X_week9 = np.array([[0.999999, 0.999999, 0.999999, 0.999999]])        # week 9
y_week9 = np.array([-55.827686352381896])

X_week10 = np.array([[0.355683, 0.539378, 0.969965, 0.296923]])        # week 10
y_week10 = np.array([-17.272166117748323])

X_week11 = np.array([[0.017062, 0.357698, 0.384798, 0.666174]])        # week 11
y_week11 = np.array([-12.420814399410862])

X_week12 = np.array([[0.233771, 0.405444, 0.820900, 0.846825]])        # week 12
y_week12 = np.array([-19.67662418731153])
# --- Combine all data ---
X = np.vstack([X_init, X_week1, X_week2, X_week3, X_week4, X_week5, X_week6, X_week7, X_week8, X_week9, X_week10, X_week11, X_week12])
y = np.hstack([y_init, y_week1, y_week2, y_week3, y_week4, y_week5, y_week6, y_week7, y_week8, y_week9, y_week10, y_week11, y_week12])


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# X, y now include ALL evaluations (including the final one)
y_trans = -y

gp = GaussianProcessRegressor(
    kernel=Matern(nu=2.5),
    alpha=1e-6,
    normalize_y=True
)

gp.fit(X, y_trans)


- Sensitivity check around the optimum
- Shows which dimensions matter
- Confirm no hidden interior optimum remains.
- After adding the final data point, the optimisation phase is complete; the remaining code should only refit the surrogate and analyse sensitivity and robustness of the discovered optimum.
- If this does not beat the corner, we’re done.

In [ ]:
corner = np.array([0.999, 0.999, 0.999, 0.999])

deltas = np.linspace(0.7, 1.0, 7)

for i in range(4):
    print(f"\nVarying dimension {i}:")
    for d in deltas:
        x = corner.copy()
        x[i] = d
        mu, sigma = gp.predict(x.reshape(1,-1), return_std=True)
        print(f"x={x}, predicted={-mu[0]:.2f}, uncertainty={sigma[0]:.3f}")

# Confirm no hidden interior optimum remains.

X_rand = np.random.uniform(0, 1, size=(2000, 4))
mu, _ = gp.predict(X_rand, return_std=True)

best_idx = np.argmax(mu)
print("Best predicted interior point:", X_rand[best_idx])
print("Predicted value:", -mu[best_idx])


In [ ]:
print("\nNext point to evaluate (6 dp):", np.round(X_rand[best_idx], 6))

# Final Result
- Best known region (full corner): ≈ −55.8
- Strong near-corner points (3 dims ≈ 1, 1 slightly lower): ≈ −53 to −54
- This point: −41.15
- That’s a large drop (≈ +14 in loss) from the optimum.
The near-corner evaluation at [0.927,0.990,0.947,0.715] resulted in a substantially weaker outcome than the full corner, confirming that the objective exhibits a steep, interaction-driven optimum rather than a broad plateau. Even a single reduced dimension leads to a large performance drop.
- possibly found the true optimum region
- Continued exploration away from the corner is not beneficial